# **Loading Images in Keras**


Estimated time needed: **45** minutes


<h1> CNN Training Pipeline </h1></s>


Convolutional neural networks (CNNs) have had great success in certain kinds of problems, such as image recognition. Data loading and preparation are important steps when it comes to working with such models. Increasingly, data augmentation is also required for more complex object recognition tasks. In this lab, we will discover various ways of loading images, as well as converting, augmenting and saving image datasets using the Keras API.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/images/cnn_image_loading_lab.drawio.png)


### What's a Digital Image?


A gray-scale digital image can be interpreted as a rectangular array of numbers. If we zoom into the region, we see the image is comprised of a rectangular grid of discrete blocks called pixels. We can represent these pixels with numbers called intensity values ranging from 0 to 255, as shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module3/L1/Gray_scale_lana.png)


Color images are a combination of red, blue, and green intensity values as shown here


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module3/L1/RGB_ape.png)


each channel has  values ranging from 0 to 255.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module3/L1/RGB_array.png)


Finally, to apply a neural network to Karis we sometimes add a batch dimension, this is just  an extra dimension. Just think of the batch dimension as an address that contains an image array.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module3/L1/batch_dim_images.png)


## **Table of Contents**

<ol>
    <li><a href="https://#Objectives">Objectives</a></li>
    <li>
        <a href="https://#Setup">Setup</a>
    </li>
    <li><a href="https://#Background">Objectives</a></li>
    <li>
        <a href="https://#Ways to load images">Ways to load images</a>
        <ol>
            <li><a href="https://#Ready-to-use toy datasets">Ready-to-use toy datasets</a></li>
            <li><a href="https://#Load individual images as PIL objects">Load individual images as PIL objects</a></li>
            <li><a href="https://#Load from directory of images">Load from directory of images</a></li>
            <li><a href="https://#Load image from URL">Load image from URL</a></li>
        </ol>
    </li>
</ol>


<h1 href="Objectives">Objectives</h1>

After completing this lab you will be able to **apply** Keras to:

*   Load and display images in multiple ways
*   Convert between array and PIL formats
*   Convert images to grayscale, augment images through transformations and save images to files


***


<h2 href="Setup">Setup</h2>


For this lab, we will be using the following libraries:

*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for mathematical operations.
*   [`Pillow`](https://pillow.readthedocs.io/en/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for image processing functions.
*   [`OpenCV`](https://docs.opencv.org/4.x/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for other image processing functions.
*   [`tensorflow`](https://www.tensorflow.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and neural network related functions.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for additional plotting tools.
*   [`keras`](https://keras.io/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for building artificial neural networks.


### Installing Required Libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run these notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda), you will need to install these libraries by removing the `#` sign before `!mamba` in the code cell below.


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
#!pip install numpy==1.22.3 matplotlib==3.5.1 tensorflow==2.9.0 opencv-python==4.5.5.62 keras

# Note: If your environment doesn't support "!mamba install", use "!pip install --user"

# RESTART YOUR KERNEL AFTERWARD AS WELL

In [ ]:
!pip3 install --upgrade tensorflow

# RESTART YOUR KERNEL AFTERWARD AS WELL

### Importing Required Libraries

*We recommend you import all required libraries in one place (here):*


In [ ]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass


import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # tensorflow INFO and WARNING messages are not printed 

In [ ]:
import random 

import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import PIL
import PIL.Image
from PIL import Image, ImageOps
import tensorflow as tf

import keras
from keras.preprocessing import image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img 
from keras.models import Model
from keras.layers import Input, Dense

import cv2

sns.set_context('notebook')
sns.set_style('white')

### Defining Helper Functions


In [ ]:
# This function will allow us to visualize four sample images from the loaded toy image dataset. 
def visualize(X_train):
    plt.rcParams['figure.figsize'] = (6,6) 

    for i in range(4):
        plt.subplot(2,2,i+1)
        num = random.randint(0, len(X_train))
        plt.imshow(X_train[num], cmap='gray', interpolation='none', vmin=0, vmax=255)
        plt.title("class {}".format(y_train[num]))
    
    plt.tight_layout()

## Background


Keras is an open-source Python library used for developing and evaluating deep learning models. It provides utilities for loading, preparing, converting, augmenting, and saving image data. In this lab, we will explore various ways of loading image datasets in Keras.


In [ ]:
# Print tensorflow version
print(tf.__version__)

# Try !pip install --upgrade tensorflow if the version printed
# is less than 2.9.0

## Ways to load images


We will look into four main ways of using image datasets in Keras:

*   Loading in a ready-to-use toy dataset from [Keras](https://keras.io/api/datasets/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01)
*   Loading individual images as PIL objects
*   Reading a directory of images on disk using `tf.keras.utils.image_dataset_from_directory`.
*   Loading image from URL


### A. Ready-to-use toy datasets


The `tf.keras.datasets` in Keras provide a few in-built image datasets that have been cleaned and are typically helpful in debugging models, or creating simple examples.

These include MNIST hand-written digits, Fashion MNIST, CIFAR10, and CIFAR100.


**MNIST hand-written digits** is a collection of 60,000 28x28 grayscale images belonging to 10 different classes, along with a test set of 10,000 images. Let us load in the MNIST hand-written digits dataset using the `load_data` function.


In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train is a uint8 numpy array of grayscale images with shapes (60000, 28, 28) from the training dataset. Its pixel values range from 0 to 255.

y_train is a uint8 numpy array of digit labels (integers in range 0-9) with shape (60000) for the training data.


In [ ]:
print(X_train.shape)
print(X_train.dtype)
print(y_train.shape)
print(y_train.dtype)

Let's visualize four random images from the training set. Only one color channel is needed to display the image as a grayscale. So for this step, we take a single color channel and display the image using the `plt.imshow()` method with `cmap` set to `gray`, `vmin` set to 0, and `vmax` set to 255.


In [ ]:
visualize(X_train)

#### Using image data with Autoencoders


As you will see in future labs, an autoencoder (AE) is a neural network where the input is the same as the output.  

To build an AE, you need three things:

* an encoding function
* a decoding function
* distance function between the amount of information loss between the compressed representation of your data and the decompressed representation, that is, a loss function

Some practical applications of AEs are data denoising and dimensionality reduction for data visualization.


We can define a size for our encoded representations, and use `Dense()` to create encoded representation of the input. Similarly, we can use `Dense()` to create lossy reconstructions of the input. As seen below, `encoded` has a shape of (None, 64) and `reconstruction` has the original input size of (None, 784).


In [ ]:
ENCODING_DIM = 64

# Encoded representations:
inputs = Input(shape=(784,)) 
encoded = Dense(ENCODING_DIM, activation="sigmoid")(inputs)

# Reconstructions:
encoded_inputs = Input(shape=(ENCODING_DIM,), name='encoding')
reconstruction = Dense(784, activation="sigmoid")(encoded_inputs)

print("Encoded Input: ", encoded.shape)
print("Reconstructed Input: ", reconstruction.shape)

Using this technique, we can instantiate three types of models:

* End-to-end AEs mapping inputs to reconstructions
* An encoder mapping inputs to the latent space
* A decoder that takes in points from latent space and output corresponding reconstructed samples


### Exercise 1 - Load Fashion MNIST


**Fashion MNIST** is another collection of 60,000 28x28 grayscale images belonging to 10 different classes, along with a test set of 10,000 images. Write code to load the Fashion MNIST dataset using  `fashion_mnist.load_data()` from `keras.datasets`.


In [ ]:
# Write your code here

(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()


X_train is a uint8 numpy array of grayscale images with shapes (60000, 28, 28) from the training dataset. Its pixel values range from 0 to 255.

y_train is a uint8 numpy array of digit labels (integers in range 0-9) with shape (60000,) for the training data.


Now print the shape and dtype of the X_train and y_train arrays.


Display a few randomly selected images as grayscale using the `visualize()` helper function.


In [ ]:
# Write your code here
# Visualize a few images from the training set

visualize(X_train) 

**CIFAR10** is a collection of 50,000 32x32 color training images labeled over 10 different categories, along with a test set of 10,000 images.


In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train is a uint8 numpy array of grayscale image data with shapes (50000, 32, 32, 3), containing the training data. Pixel values range from 0 to 255.

y_train is a uint8 NumPy array of labels (integers in range 0-9) with shape (50000, 1) for the training data.


In [ ]:
print(X_train.shape)
print(X_train.dtype)

In [ ]:
visualize(X_train)

**CIFAR100** is a collection of 50,000 32x32 color training images labeled over 100 fine-grained classes and 20 coarse-grained classes, along with a test set of 10,000 images. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (label) and a "coarse" label (superclass).


Using the fine `label_mode`:


In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data(label_mode = 'fine')

X_train is a uint8 numpy array of grayscale image data with shapes (50000, 32, 32, 3), containing the training data. Pixel values range from 0 to 255.

y_train is a uint8 numpy array of labels (integers in range 0-99) with shape (50000, 1) for the training data.


In [ ]:
print(X_train.shape)
print(X_train.dtype)

In [ ]:
visualize(X_train)

### Exercise 2 - Load CIFAR100 with coarse label_mode


Load the CIFAR100 dataset by setting the pramter ```label_mode = 'coarse'``` in the ```load_data``` function .


In [ ]:
# Write your solution here
# Load the Fashion CIFAR100 dataset using the coarse label_mode



(X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data(label_mode = 'coarse')

Visualize a few images from the training set using the `visualize()` helper function.


In [ ]:
# Write your solution here
visualize(X_train)

### B. Load individual images as PIL objects


We will save the following image into our working environment.


In [ ]:
import requests

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module3/L1/Dog_Breeds.jpg'
filename = URL.split('/')[-1]
r = requests.get(URL, allow_redirects=True)
open(filename, 'wb').write(r.content)

Next, we will specify a `target_size` of the image using a tuple of ints.


In [ ]:
img_height, img_width = 100, 100

Using the path of the image we saved, we will use the `load_img` function to return a PIL Image instance.


In [ ]:
gray_img = load_img(
    'Dog_Breeds.jpg',
    target_size=(img_height, img_width),
    interpolation='nearest', grayscale = True)

color_img = load_img(
    'Dog_Breeds.jpg',
    target_size=(img_height, img_width),
    interpolation='nearest', grayscale = False)


print(type(gray_img))
print(type(color_img))

We can plot each image 


In [ ]:
plt.imshow(gray_img,cmap="gray")
plt.show()

In [ ]:
plt.imshow(color_img)
plt.show()

#### Converting an image


We can convert the PIL image to a 3D numpy array of pixel data, and a single image to a batch using the following code:


In [ ]:
input_arr = tf.keras.preprocessing.image.img_to_array(color_img)


print("image shape",input_arr.shape)


We need to add the batch dimension before the using  Keras, we can add the batch dimension as follows:


In [ ]:
# Convert single image to a batch
input_arr_batch = np.array([input_arr])
#or
input_arr_batch=input_arr.reshape(-1,input_arr.shape[0],input_arr.shape[1],input_arr.shape[2])

In [ ]:
print("image shape plus batch dimension",input_arr_batch.shape)

Similarly, we can convert a numpy array of pixel data back to an image.


In [ ]:
# Write your solution here
# Convert the numpy array back to an image
color_img = tf.keras.preprocessing.image.array_to_img(input_arr)
plt.imshow(color_img)
plt.show()

#### Saving an image


In [ ]:
tf.keras.preprocessing.image.save_img('dog_color_img.jpg', color_img)

### C. Load from directory of images


We can use the `ImageDataGenerator` class in Keras to load train, test, and validation datasets. This is especially useful when working with datasets containing several thousand or millions of images. More details on content provided in this sub-section can be found in the Tensorflow [documentation](https://www.tensorflow.org/tutorials/load_data/images?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01).


We will first load in our dataset of flowers, and then look at what we have.

The dataset has five types of flowers.

| Category | Flower       |
| -------- | ------------ |
| 0        | `Daisies`    |
| 1        | `Dandelions` |
| 2        | `Roses`      |
| 3        | `Sunflowers` |
| 4        | `Tulips`     |

Let's load the data and take a look.

Because the flower dataset is not one of the default small toy datasets available, we'll need to download it using the `keras.utils.get_file()` function.  
Note that you will only need to specify the first three arguments.

```python
keras.utils.get_file(
    fname=None,
    origin=None,
    untar=False,
    md5_hash=None,
    file_hash=None,
    cache_subdir='datasets',
    hash_algorithm='auto',
    extract=False,
    archive_format='auto',
    cache_dir=None
)
```


In [ ]:
import pathlib
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/datasets/flower_photos.tgz"

# Download the data and track where it's saved using tf.keras.utils.get_file in a variable called data_dir
data_dir = keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)

for folder in data_dir.glob('[!LICENSE]*'):
    print('The', folder.name, 'folder has', len(list(folder.glob('*.jpg'))), 'pictures')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count, 'total images')

The files are stored in the following directory.

```python
flower_photos
    └── daisy
    └── dandelion
    └── roses
    └── sunflowers
    └── tulips

```


It seems there are a substantial amount of photos for each type of photo.

We can checkout some of the images below.


#### Visualizing images


In [ ]:
dandelion = list(data_dir.glob('dandelion/*'))
PIL.Image.open(str(dandelion[1]))

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[1]))

In [ ]:
sunflowers = list(data_dir.glob('sunflowers/*'))
PIL.Image.open(str(sunflowers[1]))

In [ ]:
daisy = list(data_dir.glob('daisy/*'))

PIL.Image.open(str(daisy[1]))

We can see that these photos are in various lighting conditions, framing, sizes, and zoom.


In [ ]:
# The batch size simply specifies the number of images to pass through our neural network at a time, until the entire training set is passed through. 32 is the default
batch_size = 32

# Here we set the size of all the images to be 200x200
img_height = 200
img_width = 200

Here we split our images into training and validation sets using the `keras.utils.image_dataset_from_directory()` function.

*   Because our flower images are sorted into subfolders named after the type of flower, this method can automatically load in images and label them with the correct classes.
*   In order to plug these photos into our neural network as an input, we'll have to standardize the image sizes. We can do that with the `image_size` argument.


Let us use `image_dataset_from_directory` to load images off disk:


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

### Exercise 3 - Load validation set from the directory


Use the `image_dataset_from_directory` to load the validation set from the directory


In [ ]:
# Write your code here

  validation_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

We can show the class names 


In [ ]:
class_names = train_ds.class_names
class_names

Let's look at some sample rows from the dataset we loaded:


In [ ]:
# .take() will take the first batch from a tensorflow dataset. 
# In our case it has taken the first 32 images.
first_batch = train_ds.take(1)

plt.figure(figsize = (25,10))
for img, lbl in first_batch:
    # lets look at the first 10 images
    for i in np.arange(10):
        plt.subplot(2,5,i+1)
        plt.imshow(img[i].numpy().astype('uint8'))
        plt.title(class_names[lbl[i]])
        plt.axis("off")

We can see that after importing our images, they're all exactly square with `200 x 200` pixels.


#### Augmenting Images


Next, we will see how we can use `ImageDataGenerator` class to create tensor image data batches that have been augmented to have certain characteristics. Image data augmentation is useful in expanding the training dataset in order to improve the performance and ability of the model to generalize. Here are some ways in which we can augment images in our dataset:


*   **rescale**: rescaling factor
*   **rotation_range**: degree of random rotations of the images
*   **width_shift_range**: upper bound for random shift, either left or right
*   **height_shift_range**: upper bound for random shift, either up or down
*   **vertical_flip**: flip the image vertically


Further details on these options can be found in the Tensorflow [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01).


In [ ]:
import tensorflow as tf

img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    vertical_flip=True,
)

flowers_data = img_gen.flow_from_directory(data_dir)
images, labels = next(flowers_data)

print(images.shape)
print(labels.shape)

### Exercise 4 - Visualizing augmented images


Visualize 10 images from the augmented tensor image data batch.


In [ ]:
# Write your code here

plt.figure(figsize=(25, 10))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(images[i])
    index = [index for index, each_item in enumerate(labels[i]) if each_item]
    plt.title(list(flowers_data.class_indices.keys())[index[0]])
    plt.axis("off")
    

### D. Load image from URL


In the _Load individual images as PIL objects section_, we saw how we can download images to our working directory and, using the path of the image we saved, use the `load_img` function to return a PIL Image instance.


We can use custom functions like the one below to load images from a URL. This function was  borrowed from [statsmaths](https://statsmaths.github.io/stat289-f18/solutions/tutorial28-keras-images.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01).


In [ ]:
def load_image(link, target_size=None):
    import requests
    import shutil
    import os
    _, ext = os.path.splitext(link)
    r = requests.get(link, stream=True)
    with open('temp.' + ext, 'wb') as f:
        r.raw.decode_content = True
        shutil.copyfileobj(r.raw, f)
    img = tf.keras.preprocessing.image.load_img('temp.' + ext, target_size=target_size)
    return tf.keras.preprocessing.image.img_to_array(img)

In [ ]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module3/L1/Dog_Breeds.jpg'

img = load_image(URL, target_size=(100, 100))
plt.imshow(img/255)